# Session 10: OOP practice

In this Session we are going to practice all the concepts we've learned in the previous sessions.

We are going to create a class called Esios that will work as an interface for the API of the Spanish Power Market.

## Exercise 1

Create a class called Esios that gets the token from the `.env` file. In order to access the token you should use the `python-dotenv` package and call the function `load_dotenv()` before trying to access the token.

It should have a private attribute called `token`, and another public attribute called `url` that will be the base url for the API.

It should also have a method called `__load_dotenv` that will load the token from the `.env` file.

In [4]:
import os
from time import sleep
from dotenv import load_dotenv
import datetime
import requests
import pandas as pd
import json
from functools import reduce


class Esios():

    def __init__(self):
        self.url = "https://api.esios.ree.es/"
        self.__load_dotenv()
        self.__token = os.getenv("API_KEY")

    def __load_dotenv(self):
        load_dotenv()


In [5]:
esios = Esios()

print(esios.url)

https://api.esios.ree.es/


In [6]:
print(esios.__token) # nope, can't access it because it's private

AttributeError: 'Esios' object has no attribute '__token'

## Exercise 2

Create a method called `__get_token` that will return the token.

In [7]:
class Esios():

    def __init__(self):
        self.url = "https://api.esios.ree.es/"
        self.__load_dotenv()
        self.__token = os.getenv("API_KEY")

    def __load_dotenv(self):
        load_dotenv()
        
    def __get_token(self):
        return self.__token

In [8]:
esios = Esios()

esios.__get_token()

AttributeError: 'Esios' object has no attribute '__get_token'

## Exercise 3

Add new methods to create the headers. The headers should be a dictionary with the keys that follow:

```python
headers = {
    'Accept': 'application/json; application/vnd.esios-api-v1+json',
    'Content-Type': 'application/json',
    'x-api-key': f'{api_token}'
}
```

In [11]:
class Esios():

    def __init__(self):
        self.url = "https://api.esios.ree.es/"
        self.__load_dotenv()
        self.__token = os.getenv("API_KEY")

    def __load_dotenv(self):
        load_dotenv()
        
    def __get_token(self):
        return self.__token
    
    def __create_headers(self):
        return {
            'Accept': 'application/json; application/vnd.esios-api-v1+json',
            'Content-Type': 'application/json',
            'x-api-key': f'{self.__token}'
        }

In [12]:
esios = Esios()

headers = esios.__create_headers()

print(headers['Accept'])
print(headers['Content-Type'])
print(headers['x-api-key'][:5])

AttributeError: 'Esios' object has no attribute '__create_headers'

## Exercise 4

Now add a method that takes in two datetimes, in a specific format and returns the parameters dictionary that will be used to make the request to the API.

The format of the datetimes should be `YYYY-MM-DDTHH:MM:SSZ`. This format should be an attribute of the class, so that we can check the format of the datetimes, and also change it if needed.

The parameters dictionary should have the following keys:

```python
parameters = {
    'start_date': start_datetime,
    'end_date': end_datetime,
}
```

In [31]:
class Esios():

    def __init__(self):
        self.url = "https://api.esios.ree.es/"
        self.__load_dotenv()
        self.__token = os.getenv("API_KEY")

    def __load_dotenv(self):
        load_dotenv()
        
    def __get_token(self):
        return self.__token
    
    def __create_headers(self):
        return {
            'Accept': 'application/json; application/vnd.esios-api-v1+json',
            'Content-Type': 'application/json',
            'x-api-key': f'{self.__token}'
        }
        
        
    def create_params_dictionary(self, start, end):
        start_datetime = pd.to_datetime(start)
        end_datetime = pd.to_datetime(end)
        return {
            'start': start_datetime,
            'end': end_datetime
        }

In [32]:
esios = Esios()

# `YYYY-MM-DDTHH:MM:SSZ`

params = esios.create_params_dictionary('2025-01-01', '2025-01-02')

params['start']

Timestamp('2025-01-01 00:00:00')

## Exercise 5

Now we have access to the token, the headers, and parameters, add a method called `get_available_indicators` that will return the available indicators from the API.

The method should return the raw response from the API, in JSON format.

In [37]:
# https://api.esios.ree.es/indicators

class Esios():

    def __init__(self):
        self.url = "https://api.esios.ree.es/"
        self.__load_dotenv()
        self.__token = os.getenv("API_KEY")

    def __load_dotenv(self):
        load_dotenv()
        
    def __get_token(self):
        return self.__token
    
    def __create_headers(self):
        return {
            'Accept': 'application/json; application/vnd.esios-api-v1+json',
            'Content-Type': 'application/json',
            'x-api-key': f'{self.__token}'
        }
        
        
    def create_params_dictionary(self, start, end):
        start_datetime = pd.to_datetime(start)
        end_datetime = pd.to_datetime(end)
        return {
            'start': start_datetime,
            'end': end_datetime
        }
        
    def get_available_indicators(self):
        endpoint_url = self.url + 'indicators'
        headers = self.__create_headers()
        response = requests.get(url=endpoint_url, headers=headers)
        return response.json()

In [41]:
esios = Esios()

ind_json = esios.get_available_indicators()

ind_json['indicators']

[{'name': 'Generación programada PBF Hidráulica UGH',
  'description': '<p>Es el programa de energía diario, con desglose horario, de las diferentes Unidades de Programación correspondientes a ventas y adquisiciones de energía en el sistema eléctrico peninsular español. En concreto este indicador se refiere a las unidades de programación con tipo de producción hidráulica UGH.</p><p>Este programa es establecido por el OS a partir de la casación del OM y de las nominaciones de programas de todas y cada una de las Unidades de Programación que le han sido comunicadas por los sujetos titulares de dichas Unidades de Programación, incluyendo las correspondientes a la ejecución de contratos bilaterales con entrega física de los cuales se ha confirmado la ejecución.</p><p><b>Publicación:</b> diariamente a partir de las 13:45 horas con la información del día D+1.</p>',
  'short_name': 'Hidráulica UGH',
  'id': 1},
 {'name': 'Generación programada PBF Hidráulica no UGH',
  'description': '<p>Es e

## Exercise 6

Add a new method called `save_indicators` that will save the indicators to a file called `indicators.json`. Give it a parameter for the user to decide if they want to save the indicators as JSON or CSV, and save the file accordingly.

In [43]:
def my_sum(a, b):
    return to_int(a) + to_int(b)

def to_int(a):
    return int(a)

my_sum('4', '5')

9

In [46]:
class Esios():

    def __init__(self):
        self.url = "https://api.esios.ree.es/"
        self.__load_dotenv()
        self.__token = os.getenv("API_KEY")

    def __load_dotenv(self):
        load_dotenv()
        
    def __get_token(self):
        return self.__token
    
    def __create_headers(self):
        return {
            'Accept': 'application/json; application/vnd.esios-api-v1+json',
            'Content-Type': 'application/json',
            'x-api-key': f'{self.__token}'
        }
        
        
    def create_params_dictionary(self, start, end):
        start_datetime = pd.to_datetime(start)
        end_datetime = pd.to_datetime(end)
        return {
            'start': start_datetime,
            'end': end_datetime
        }
        
    def get_available_indicators(self, save=False, extension='json'):
        endpoint_url = self.url + 'indicators'
        headers = self.__create_headers()
        response = requests.get(url=endpoint_url, headers=headers)
        if save:
            self.save_indicators(response.json(), extension)
    
    def save_indicators(self, json_file, extension='json'):
        if extension == 'csv':
            pd.DataFrame(json_file['indicators']).to_csv()
        elif extension == 'json':
            with open(json_file, 'w') as file:
                json.dump(json_file, file, indent=4)
        else:
            print('That format is not accepted')
            pass
        print(f'The indicators were saved as `{extension}`')
                

In [47]:
esios = Esios()

esios.get_available_indicators(extension='csv', save=True)

The indicators were saved as `csv`


## Exercise 7

Now that we have the indicators list, let's create a method called `get_indicator_data` that will get the data for a specific indicator between two dates.

Also, add a method to save the indicator's data to a file called `indicator_data.json` or `indicator_data.csv` depending on the user's choice.

In [52]:
class Esios():

    def __init__(self):
        self.url = "https://api.esios.ree.es/"
        self.__load_dotenv()
        self.__token = os.getenv("API_KEY")

    def __load_dotenv(self):
        load_dotenv()
        
    def __get_token(self):
        return self.__token
    
    def __create_headers(self):
        return {
            'Accept': 'application/json; application/vnd.esios-api-v1+json',
            'Content-Type': 'application/json',
            'x-api-key': f'{self.__token}'
        }
        
        
    def create_params_dictionary(self, start, end):
        start_datetime = pd.to_datetime(start)
        end_datetime = pd.to_datetime(end)
        return {
            'start': start_datetime,
            'end': end_datetime
        }
        
    def get_available_indicators(self, save=False, extension='json'):
        endpoint_url = self.url + 'indicators'
        headers = self.__create_headers()
        response = requests.get(url=endpoint_url, headers=headers)
        if save:
            self.save_indicators(response.json(), extension)
    
    def save_indicators(self, json_file, extension='json'):
        if extension == 'csv':
            pd.DataFrame(json_file['indicators']).to_csv()
        elif extension == 'json':
            with open(json_file, 'w') as file:
                json.dump(json_file, file, indent=4)
        else:
            print('That format is not accepted')
            pass
        print(f'The indicators were saved as `{extension}`')
                
    def get_indicator_data(self, indicator_id, start, end):
        request_url = self.url + f'indicators/{indicator_id}'
        headers = self.__create_headers()
        params = self.create_params_dictionary(start, end)
        response = requests.get(url=request_url, params=params, headers=headers)
        return response.json()
        

In [53]:
esios = Esios()

esios.get_indicator_data(4, '2025-01-01', '2025-01-02')

{'indicator': {'name': 'Generación programada PBF Nuclear',
  'short_name': 'Nuclear',
  'id': 4,
  'composited': False,
  'step_type': 'step',
  'disaggregated': False,
  'magnitud': [{'name': 'Energía', 'id': 13}],
  'tiempo': [{'name': 'Hora', 'id': 4}],
  'geos': [{'geo_id': 8741, 'geo_name': 'Península'}],
  'values_updated_at': '2025-02-05T14:30:45.000+01:00',
  'values': [{'value': 7094.9,
    'datetime': '2025-02-06T00:00:00.000+01:00',
    'datetime_utc': '2025-02-05T23:00:00Z',
    'tz_time': '2025-02-05T23:00:00.000Z',
    'geo_id': 8741,
    'geo_name': 'Península'},
   {'value': 7094.9,
    'datetime': '2025-02-06T01:00:00.000+01:00',
    'datetime_utc': '2025-02-06T00:00:00Z',
    'tz_time': '2025-02-06T00:00:00.000Z',
    'geo_id': 8741,
    'geo_name': 'Península'},
   {'value': 7094.9,
    'datetime': '2025-02-06T02:00:00.000+01:00',
    'datetime_utc': '2025-02-06T01:00:00Z',
    'tz_time': '2025-02-06T01:00:00.000Z',
    'geo_id': 8741,
    'geo_name': 'Península'},


## Exercise 8

Ok, we have everything now. Let's create a method called `get_price_data` that will get the price data for a specific date. Using the list of indicators, find the id of the indicator that corresponds to the price data, knowing that the name of the indicator is 'Precio mercado SPOT Diario'.

In [ ]:
class Esios():

    def __init__(self):
        self.url = "https://api.esios.ree.es/"
        self.__load_dotenv()
        self.__token = os.getenv("API_KEY")

    def __load_dotenv(self):
        load_dotenv()
        
    def __get_token(self):
        return self.__token
    
    def __create_headers(self):
        return {
            'Accept': 'application/json; application/vnd.esios-api-v1+json',
            'Content-Type': 'application/json',
            'x-api-key': f'{self.__token}'
        }
        
        
    def create_params_dictionary(self, start, end):
        start_datetime = pd.to_datetime(start)
        end_datetime = pd.to_datetime(end)
        return {
            'start': start_datetime,
            'end': end_datetime
        }
        
    def get_available_indicators(self, save=False, extension='json'):
        endpoint_url = self.url + 'indicators'
        headers = self.__create_headers()
        response = requests.get(url=endpoint_url, headers=headers)
        if save:
            self.save_indicators(response.json(), extension)
    
    def save_indicators(self, json_file, extension='json'):
        if extension == 'csv':
            pd.DataFrame(json_file['indicators']).to_csv()
        elif extension == 'json':
            with open(json_file, 'w') as file:
                json.dump(json_file, file, indent=4)
        else:
            print('That format is not accepted')
            pass
        print(f'The indicators were saved as `{extension}`')
                
    def get_indicator_data(self, indicator_id, start, end):
        request_url = self.url + f'indicators/{indicator_id}'
        headers = self.__create_headers()
        params = self.create_params_dictionary(start, end)
        response = requests.get(url=request_url, params=params, headers=headers)
        return response.json()
    
    def get_price_data(self, start, end):
        return self.get_indicator_data(600, start, end)
        

## Exercise 9

Oops, it is bringing data for every country!

Update the constructor so that an attribute called `allowed_geo_ids` is added. This attribute should be a list of the `geo_id` that we want to get the data from.

Then, update the method `get_indicator_data` so that it only gets the data for the `geo_id` that is in the `allowed_geo_ids` list.

The relation between the country's `geo_name` and the `geo_id` is the following:

```python
{
    1: 'Portugal',
    2: 'Francia',
    3: 'España',
    8826: 'Alemania',
    8827: 'Bélgica',
    8828: 'Países Bajos'
}
```


In [ ]:
class Esios():

    def __init__(self):
        self.url = "https://api.esios.ree.es/"
        self.__load_dotenv()
        self.__token = os.getenv("API_KEY")
        self.allowed_geo_ids = [{
            1: 'Portugal',
            2: 'Francia',
            3: 'España',
            8826: 'Alemania',
            8827: 'Bélgica',
            8828: 'Países Bajos'
            }]
        

    def __load_dotenv(self):
        load_dotenv()
        
    def __get_token(self):
        return self.__token
    
    def __create_headers(self):
        return {
            'Accept': 'application/json; application/vnd.esios-api-v1+json',
            'Content-Type': 'application/json',
            'x-api-key': f'{self.__token}'
        }
        
        
    def create_params_dictionary(self, start, end):
        start_datetime = pd.to_datetime(start)
        end_datetime = pd.to_datetime(end)
        return {
            'start': start_datetime,
            'end': end_datetime
        }
        
    def get_available_indicators(self, save=False, extension='json'):
        endpoint_url = self.url + 'indicators'
        headers = self.__create_headers()
        response = requests.get(url=endpoint_url, headers=headers)
        if save:
            self.save_indicators(response.json(), extension)
    
    def save_indicators(self, json_file, extension='json'):
        if extension == 'csv':
            pd.DataFrame(json_file['indicators']).to_csv()
        elif extension == 'json':
            with open(json_file, 'w') as file:
                json.dump(json_file, file, indent=4)
        else:
            print('That format is not accepted')
            pass
        print(f'The indicators were saved as `{extension}`')
                
    def get_indicator_data(self, indicator_id, start, end):
        if indicator_id in self.allowed_geo_ids:  
            request_url = self.url + f'indicators/{indicator_id}'
            headers = self.__create_headers()
            params = self.create_params_dictionary(start, end)
            response = requests.get(url=request_url, params=params, headers=headers)
            return response.json()
        else:
            print('That indicator is not allowed')
        
    
    def get_price_data(self, start, end):
        return self.get_indicator_data(600, start, end)

## Exercise 10

Create a method called `get_indicator_name` that will return the name of the indicator given the `indicator_id` that you pass as a parameter.

Use the `indicators.json` file to get the name of the indicator, and if the file does not exist, call the method `get_available_indicators` to get the indicators.

In [ ]:
class Esios():

    def __init__(self):
        self.url = "https://api.esios.ree.es/"
        self.__load_dotenv()
        self.__token = os.getenv("API_KEY")
        self.allowed_geo_ids = [{
            1: 'Portugal',
            2: 'Francia',
            3: 'España',
            8826: 'Alemania',
            8827: 'Bélgica',
            8828: 'Países Bajos'
            }]
        

    def __load_dotenv(self):
        load_dotenv()
        
    def __get_token(self):
        return self.__token
    
    def __create_headers(self):
        return {
            'Accept': 'application/json; application/vnd.esios-api-v1+json',
            'Content-Type': 'application/json',
            'x-api-key': f'{self.__token}'
        }
        
        
    def create_params_dictionary(self, start, end):
        start_datetime = pd.to_datetime(start)
        end_datetime = pd.to_datetime(end)
        return {
            'start': start_datetime,
            'end': end_datetime
        }
        
    def get_available_indicators(self, save=False, extension='json'):
        endpoint_url = self.url + 'indicators'
        headers = self.__create_headers()
        response = requests.get(url=endpoint_url, headers=headers)
        if save:
            self.save_indicators(response.json(), extension)
    
    def save_indicators(self, json_file, extension='json'):
        if extension == 'csv':
            pd.DataFrame(json_file['indicators']).to_csv()
        elif extension == 'json':
            with open(json_file, 'w') as file:
                json.dump(json_file, file, indent=4)
        else:
            print('That format is not accepted')
            pass
        print(f'The indicators were saved as `{extension}`')
                
    def get_indicator_data(self, indicator_id, start, end):
        if indicator_id in self.allowed_geo_ids:  
            request_url = self.url + f'indicators/{indicator_id}'
            headers = self.__create_headers()
            params = self.create_params_dictionary(start, end)
            response = requests.get(url=request_url, params=params, headers=headers)
            return response.json()
        else:
            print('That indicator is not allowed')
        
    
    def get_price_data(self, start, end):
        return self.get_indicator_data(600, start, end)
    
    def get_indicator_name(self, indicator_id):
        try:
            with open('indicators.json', 'r') as file:
                indicators = json.load(file)
        except FileNotFoundError:
            indicators = self.get_available_indicators(save=True)
        
        for indicator in indicators['indicators']:
            if indicator['id'] == indicator_id:
                return indicator['name']
        return None

## Exercise 11

Update the `get_indicator_data` method so that it prints the name of the indicator that it is getting the data from.

In [ ]:
class Esios():

    def __init__(self):
        self.url = "https://api.esios.ree.es/"
        self.__load_dotenv()
        self.__token = os.getenv("API_KEY")
        self.allowed_geo_ids = [{
            1: 'Portugal',
            2: 'Francia',
            3: 'España',
            8826: 'Alemania',
            8827: 'Bélgica',
            8828: 'Países Bajos'
            }]
        

    def __load_dotenv(self):
        load_dotenv()
        
    def __get_token(self):
        return self.__token
    
    def __create_headers(self):
        return {
            'Accept': 'application/json; application/vnd.esios-api-v1+json',
            'Content-Type': 'application/json',
            'x-api-key': f'{self.__token}'
        }
        
        
    def create_params_dictionary(self, start, end):
        start_datetime = pd.to_datetime(start)
        end_datetime = pd.to_datetime(end)
        return {
            'start': start_datetime,
            'end': end_datetime
        }
        
    def get_available_indicators(self, save=False, extension='json'):
        endpoint_url = self.url + 'indicators'
        headers = self.__create_headers()
        response = requests.get(url=endpoint_url, headers=headers)
        if save:
            self.save_indicators(response.json(), extension)
    
    def save_indicators(self, json_file, extension='json'):
        if extension == 'csv':
            pd.DataFrame(json_file['indicators']).to_csv()
        elif extension == 'json':
            with open(json_file, 'w') as file:
                json.dump(json_file, file, indent=4)
        else:
            print('That format is not accepted')
            pass
        print(f'The indicators were saved as `{extension}`')
                
    def get_indicator_data(self, indicator_id, start, end):
        indicator_name = self.get_indicator_name(indicator_id)
        print(f"Getting data for indicator: {indicator_name}")
        if indicator_id in self.allowed_geo_ids:  
            request_url = self.url + f'indicators/{indicator_id}'
            headers = self.__create_headers()
            params = self.create_params_dictionary(start, end)
            response = requests.get(url=request_url, params=params, headers=headers)
            return response.json()
        else:
            print('That indicator is not allowed')
        
    
    def get_price_data(self, start, end):
        return self.get_indicator_data(600, start, end)
    
    def get_indicator_name(self, indicator_id):
        try:
            with open('indicators.json', 'r') as file:
                indicators = json.load(file)
        except FileNotFoundError:
            indicators = self.get_available_indicators(save=True)
        
        for indicator in indicators['indicators']:
            if indicator['id'] == indicator_id:
                return indicator['name']
        return None

Getting data for indicator 460: Previsión diaria de la demanda eléctrica peninsular


{'indicator': {'name': 'Previsión diaria de la demanda eléctrica peninsular',
  'short_name': 'Previsión diaria',
  'id': 460,
  'composited': False,
  'step_type': 'step',
  'disaggregated': False,
  'magnitud': [{'name': 'Potencia', 'id': 20}],
  'tiempo': [{'name': 'Quince minutos', 'id': 218}],
  'geos': [{'geo_id': 8741, 'geo_name': 'Península'}],
  'values_updated_at': '2024-01-01T23:28:15.000+01:00',
  'values': []}}

## Exercise 12

Create a method that receives a list of `indicator_id` and returns a dataframe with the data for all the indicators in the list.

You should use `pd.merge` to merge the dataframes, and call the method `get_indicator_name` to get the name of the indicator as the name of the column.

Use for eaach indicator the method `get_indicator_data` to get the data.

In [ ]:
from functools import reduce

class Esios():

    def __init__(self):
        self.url = "https://api.esios.ree.es/"
        self.__load_dotenv()
        self.__token = os.getenv("API_KEY")
        self.allowed_geo_ids = [{
            1: 'Portugal',
            2: 'Francia',
            3: 'España',
            8826: 'Alemania',
            8827: 'Bélgica',
            8828: 'Países Bajos'
            }]
        

    def __load_dotenv(self):
        load_dotenv()
        
    def __get_token(self):
        return self.__token
    
    def __create_headers(self):
        return {
            'Accept': 'application/json; application/vnd.esios-api-v1+json',
            'Content-Type': 'application/json',
            'x-api-key': f'{self.__token}'
        }
        
        
    def create_params_dictionary(self, start, end):
        start_datetime = pd.to_datetime(start)
        end_datetime = pd.to_datetime(end)
        return {
            'start': start_datetime,
            'end': end_datetime
        }
        
    def get_available_indicators(self, save=False, extension='json'):
        endpoint_url = self.url + 'indicators'
        headers = self.__create_headers()
        response = requests.get(url=endpoint_url, headers=headers)
        if save:
            self.save_indicators(response.json(), extension)
    
    def save_indicators(self, json_file, extension='json'):
        if extension == 'csv':
            pd.DataFrame(json_file['indicators']).to_csv()
        elif extension == 'json':
            with open(json_file, 'w') as file:
                json.dump(json_file, file, indent=4)
        else:
            print('That format is not accepted')
            pass
        print(f'The indicators were saved as `{extension}`')
                
    def get_indicator_data(self, indicator_id, start, end):
        indicator_name = self.get_indicator_name(indicator_id)
        print(f"Getting data for indicator: {indicator_name}")
        if indicator_id in self.allowed_geo_ids:  
            request_url = self.url + f'indicators/{indicator_id}'
            headers = self.__create_headers()
            params = self.create_params_dictionary(start, end)
            response = requests.get(url=request_url, params=params, headers=headers)
            return response.json()
        else:
            print('That indicator is not allowed')
        
    
    def get_price_data(self, start, end):
        return self.get_indicator_data(600, start, end)
    
    def get_indicator_name(self, indicator_id):
        try:
            with open('indicators.json', 'r') as file:
                indicators = json.load(file)
        except FileNotFoundError:
            indicators = self.get_available_indicators(save=True)
        
        for indicator in indicators['indicators']:
            if indicator['id'] == indicator_id:
                return indicator['name']
        return None
    
    def get_indicators_data(self, indicator_ids, start, end):
        dataframes = []
        for indicator_id in indicator_ids:
            data = self.get_indicator_data(indicator_id, start, end)
            df = pd.DataFrame(data['indicator']['values'])
            df = df.rename(columns={'value': self.get_indicator_name(indicator_id)})
            dataframes.append(df)
        
        merged_df = reduce(lambda left, right: pd.merge(left, right, on='datetime', how='outer'), dataframes)
        return merged_df